# Dados sobre os clubes nesse período da Libertadores.

In [1]:
import pandas as pd


In [2]:
df = pd.read_csv("../../../data/raw/libertadores-results-ds.csv")
df.head()

,Edition,Round,Date,Home Club,Away Club,Home Score,AwayScore
0,2023,Final,4/11/2023,Fluminense FC,Boca Juniors,2,1
1,2023,Semifinal,6/10/2023,Palmeiras,Boca Juniors,1,1
2,2023,Semifinal,5/10/2023,Internacional,Fluminense FC,1,2
3,2023,Semifinal,29/09/2023,Boca Juniors,Palmeiras,0,0
4,2023,Semifinal,28/09/2023,Fluminense FC,Internacional,2,2


# Vendo a quantidade total de jogos do dataset

In [3]:
len(df)

1954

# Vendo a quantidade diferente de times no data set

In [4]:
# Contagem de times diferentes no dataset
# Um time joga no MINIMO 1 jogo em casa, por isso pode usar essa coluna
df['Home Club'].nunique()


132

# Vendo times com mais jogos

In [5]:
df['Home Club'].value_counts()

Home Club
River Plate            62
Boca Juniors           57
Nacional               54
Palmeiras              54
Atlético Mineiro       54
                       ..
Deportivo Petare        1
Everton CD              1
Deportivo Capiatá       1
Club Puebla             1
Sportivo Trinidense     1
Name: count, Length: 132, dtype: int64

# Se a Libertadores fosse pontos corridos?
### Temos esse esquema no Campeonato Paulista, onde mesmo que há a divisão dos grupos e posteriormente a fase mata-mata, há uma tabela de classificação geral
### Vou recriar essa ideia, agora na Copa Libertadores
### Será por separado por ANO

In [6]:
def statusMandante(dfAno):
    
    dfAno = dfAno.copy()

    dfAno['Vitória Casa'] = dfAno['Home Score'] > dfAno['AwayScore']
    dfAno['Empate'] = dfAno['Home Score'] == dfAno['AwayScore']
    dfAno['Derrota Casa'] = dfAno['Home Score'] < dfAno['AwayScore']

    statusMandante = dfAno.groupby('Home Club').agg({
        'Vitória Casa': 'sum',
        'Empate': 'sum',          # corrigido aqui
        'Derrota Casa': 'sum',
        'Home Score': 'sum',
        'AwayScore': 'sum',
        'Home Club': 'count'
    }).rename(columns={
        'Vitória Casa': 'Vitórias',
        'Empate': 'Empates',
        'Derrota Casa': 'Derrotas',
        'Home Score': 'Gols Pró',
        'AwayScore': 'Gols Sofridos',
        'Home Club': 'Jogos'
    })

    return statusMandante


In [7]:
def statusVisitante(dfAno):

    dfAno = dfAno.copy()  # evita warning ao criar colunas

    dfAno['Vitória Visitante'] = dfAno['Home Score'] < dfAno['AwayScore']
    dfAno['Empate'] = dfAno['Home Score'] == dfAno['AwayScore']
    dfAno['Derrota Visitante'] = dfAno['Home Score'] > dfAno['AwayScore']

    statusVisitante = dfAno.groupby('Away Club').agg({
        'Vitória Visitante': 'sum',
        'Empate': 'sum',
        'Derrota Visitante': 'sum',
        'Home Score': 'sum',
        'AwayScore': 'sum',
        'Away Club': 'count' 
    }).rename(columns={
        'Vitória Visitante': 'Vitórias',
        'Empate': 'Empates',
        'Derrota Visitante': 'Derrotas',
        'Home Score': 'Gols Pró',
        'AwayScore': 'Gols Sofridos',
        'Away Club': 'Jogos'
    })

    return statusVisitante

In [8]:
def criarTabelaClassificacao(df, ano):
    anoTabela = df[df['Edition'] == ano]

    tabelaMandantes = statusMandante(anoTabela)
    tabelaVisitantes = statusVisitante(anoTabela)

    # Concatena as tabelas de mandantes e visitantes para ter todos os dados em uma única tabela
    # Agrupa os dados por time (index) e soma os valores para ter o total de cada estatística
    tabelaGeral = pd.concat([tabelaMandantes, tabelaVisitantes])
    tabelaGeral = tabelaGeral.groupby(tabelaGeral.index).sum()

    tabelaGeral['Saldos Gols'] = tabelaGeral['Gols Pró'] - tabelaGeral['Gols Sofridos']
    tabelaGeral['Pontos'] = tabelaGeral['Vitórias'] * 3 + tabelaGeral['Empates']
    tabelaGeral['Aproveitamento'] = tabelaGeral['Pontos'] / (tabelaGeral['Jogos'] * 3) * 100
    tabelaGeral['Aproveitamento'] = tabelaGeral['Aproveitamento'].round(2)
    tabelaGeral = tabelaGeral.sort_values(by='Pontos', ascending=False).reset_index()
    tabelaGeral = tabelaGeral.rename(columns={'index': 'Clube'})
    
    return tabelaGeral

criarTabelaClassificacao(df, 2019)


,Clube,Vitórias,Empates,Derrotas,Gols Pró,Gols Sofridos,Jogos,Saldos Gols,Pontos,Aproveitamento
0,Boca Juniors,7,3,2,19,7,12,12,24,66.67
1,Flamengo,7,3,3,25,9,13,16,24,61.54
2,Palmeiras,7,1,2,15,11,10,4,22,73.33
3,Internacional,6,3,1,14,10,10,4,21,70.00
4,Grêmio,6,2,4,16,12,12,4,20,55.56
5,River Plate,4,7,2,15,10,13,5,19,48.72
6,Cerro Porteño,5,3,2,15,7,10,8,18,60.00
7,Cruzeiro,5,2,1,7,6,8,1,17,70.83
8,LDU de Quito,4,3,3,19,10,10,9,15,50.00
9,Nacional,4,1,3,6,4,8,2,13,54.17


## Exportando resultado para futuras visualizações em alguma ferramenta de BI

In [9]:
tabelas = []

for ano in df['Edition'].unique():
    tabela = criarTabelaClassificacao(df, ano)
    tabela['Edicao'] = ano
    tabelas.append(tabela)

tabela_final = pd.concat(tabelas, ignore_index=True)

tabela_final.to_csv('../../../data/trusted/libertadores-historico.csv', index=False)
